In [2]:
import numpy as np
from matplotlib import pyplot as plt

from importlib import reload
import src.probability
reload(src.probability)
reload(src.multiplier)
reload(src.observations)
from src.probability import Prior, Posterior
from src.multiplier import FourierMultiplier
from src.forward import Heat
from src.observations import PointObservation, DiagObservation


%matplotlib

Using matplotlib backend: QtAgg


In [55]:
sig = 5e-2

N = 200
L = 1
time = 3e-2
alpha = 0.6
gamma = -1.1
delta = 0.5

## DST /DCT - discrete (co)sine transform. 
## Corresponds to a homogeneous Dirichlet/ Neumann boundary condition.
## Clusterization also works with Neumann boundary, I just chose to present 
## Dirichlet boundary
dst, dct = {}, {}
for transform in  ['dst', 'dct']:
    fwd = Heat(N=N, L=L, transform=transform, alpha=alpha, time=time)
    pr = Prior(N=N, L=L, transform=transform, gamma=gamma, delta=delta)
    post = Posterior(fwd=fwd,
                     prior=pr,
                     sigSqr=sig**2,
                     L=L,
                     N=N,
                     transform=transform)
    dic = dst if transform == 'dst' else dct
    for m in range(2, 6):
        res = post.optimize(m=m, n_iterations=250)
        design = res['x']
        dic[m] = design
        print(transform, m, design)


dst 2 [0.30716777 0.68783224]
dst 3 [0.24865751 0.4975     0.74634249]
dst 4 [0.24124854 0.46399709 0.73733744 0.73733744]
dst 5 [0.24876531 0.24876531 0.49749999 0.74623469 0.74623471]
dct 2 [0.         0.99749999]
dct 3 [0.         0.49750713 0.99749999]
dct 4 [0.         0.3671769  0.62783586 0.9975    ]
dct 5 [0.         0.36320209 0.61901755 0.9975     0.9975    ]


In [102]:
plt.close('all')

# dst = {2: array([0.30716777, 0.68783224]), 
#        3: array([0.24865751, 0.4975    , 0.74634249]), 
#        4: array([0.24124854, 0.46399709, 0.73733744, 0.73733744]),
#        5: array([0.24876531, 0.24876531, 0.49749999, 0.74623469, 0.74623471])}

fig, ax = plt.subplots(figsize=(10,5))

for m, array in dst.items():
    vals = np.repeat(m, len(array))
    ax.scatter(array, vals, s=0)

    for i, val in enumerate(array):
        ax.annotate(str(i+1), xy=(val, m+np.random.randn()*0), ha='center', va='center', fontsize=20)
            
ax.set_xlabel('Measurement Location', fontsize=22)
ax.set_ylabel('No. of Measuremnts', fontsize=22)
plt.tight_layout()

plt.savefig("latex/example.eps")

In [98]:
print(dst)

{2: array([0.30716777, 0.68783224]), 3: array([0.24865751, 0.4975    , 0.74634249]), 4: array([0.24124854, 0.46399709, 0.73733744, 0.73733744]), 5: array([0.24876531, 0.24876531, 0.49749999, 0.74623469, 0.74623471])}
